# Import thư viện

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Softmax, Dropout
from nltk.corpus import stopwords
import re
from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from keras.models import model_from_json
from keras.models import load_model

# Nhập thông tin phim

In [50]:
# Tên phim
title_film = input('Nhập tên phim')

# Thể loại phim
genre_film = input('Thể loại phim:')

# Rating phim
rating_film = input('Rating phim:')


# Tóm tắt phim
synopsis_film = input('Tóm tắt phim:')

# Tính điểm số sentiment

## Crawl tweet về phim trên twitter dựa vào tên phim

In [67]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
# options.add_argument('--incognito')
options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path='C:\chromedriver', options=options)
driver.implicitly_wait(0)
actions = ActionChains(driver)

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_276\397742964.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:\chromedriver', options=options)


In [52]:
url = "https://twitter.com/login"
driver.get(url)

In [53]:
element = WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

sleep(1)
driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="username"]').send_keys('cap246k291@gmail.com')
actions.send_keys(Keys.ENTER)
actions.perform()

sleep(5)
try:
    driver.find_element(By.CSS_SELECTOR, 'input[autocapitalize="none"]').send_keys('@cap246k291')
    actions.send_keys(Keys.ENTER)
    actions.perform()
except:
    pass

sleep(5)

driver.find_element(By.CSS_SELECTOR, 'input[autocomplete="current-password"]').send_keys('cap2@2023')
actions.send_keys(Keys.ENTER)
actions.perform()

sleep(10)

In [55]:
driver.find_element(By.CSS_SELECTOR, 'input[aria-label="Search query"]').send_keys('(#%s) lang:en'%title_film)
actions.send_keys(Keys.ENTER)
actions.perform()

sleep(2)
driver.find_elements(By.CSS_SELECTOR, 'div[role="presentation"]')[2].click()

list_tweet = []

while len(list(set(list_tweet))) < 100:
    element = WebDriverWait(driver, 600).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[aria-label="Timeline: Search timeline"]')))
    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
    soup = BeautifulSoup(html_of_interest, 'lxml')

    rv = soup.select('div[data-testid="tweetText"]')
    for i in rv:
        list_tweet.append(i.text)
    
    a = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')
    a[-1].location_once_scrolled_into_view

    sleep(3)

driver.close()

In [56]:
data_tweet = pd.DataFrame(list(set(list_tweet)), columns=['Tweet'])

In [57]:
df_imdb = pd.read_csv('..\Sentiment Reviews Film\Reviews IMDB Dataset.csv')

X_train, X_test, y_train, y_test = train_test_split(df_imdb['review'], df_imdb['sentiment'], test_size=0.2, random_state=42)

text_clsf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
text_clsf.fit(X_train,y_train)

# Tính điểm số sentiment
sentiment_score = (text_clsf.predict(data_tweet['Tweet'].values).tolist().count('positive')/len(data_tweet['Tweet']))*100

In [58]:
sentiment_score

69.0

## Dự đoán điểm số hồi quy

In [59]:
# Load model predict Rating đã train
json_file = open('predict_score_film.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("predict_score_film.h5")

In [60]:
# Hàm tiền xử lí dữ liệu text
def preprocess_text(text):
    text = text.lower() # Chuyển văn bản thành chữ thường
    text = re.sub(r'[^\w\s]',' ',text) # Xóa dấu câu
    text = re.sub("\d+", " ", text) # Xóa chữ số
    stop = stopwords.words('english') # Xóa các từ không có nghĩa (Stop Words)
    text = " ".join(text for text in text.split() if text not in stop)
    return text

In [61]:
data = pd.read_csv('E:\Crawl_Web\Rotten Tomatoes\Preprocessing\movie_cleaned.csv')[:20000]
data['Info'] = data['Genre'] + ' ' + data['Rating'] + ' ' + data['Synopsis']
data['Info'] = data['Info'].apply(preprocess_text)

vectorizer = TfidfVectorizer()
vectorizer.fit(data['Info'])

info_film = genre_film + ' ' + rating_film + ' ' + synopsis_film

vector_info_film = vectorizer.transform([preprocess_text(info_film)])

reg_score = loaded_model.predict(vector_info_film.todense())[0][0]

1/1 [==============================] - 0s 65ms/step


In [62]:
reg_score

9.088625

## Tính điểm số trung bình

In [63]:
avg_score = round((sentiment_score/10 + reg_score)/2, 1)

In [64]:
avg_score

8.0

In [65]:
if avg_score < 5:
    success = 'Flop'
elif 5 <= avg_score < 7:
    success = 'Average'
elif 7 <= avg_score < 9:
    success = 'Hit'
else:
    success = 'Super hit'

In [66]:
print('Mức độ thành công của phim là:', success)

Mức độ thành công của phim là: Hit
